In [5]:
import pandas as pd
import os
import time
import logging
from sqlalchemy import create_engine

logging.basicConfig(
    filename='logs/ingestion_db.log',
    filemode='a',
    level=logging.DEBUG,
    format='%(asctime)s %(levelname)-8s %(message)s'
)
# Database connection parameters
username = 'root'
password = '3337'
host     = 'localhost'
port     = 3306
database = 'ecom'

# Build SQLAlchemy engine
connection_string = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
conn = create_engine(connection_string)

def ingest_db(df, table_name, engine):
    """
    Ingests the given DataFrame into the specified table.
    If the table already exists, it will be replaced.
    """
    df.to_sql(table_name, con=conn, if_exists="replace", index=False)

def load_raw_data():
    """
    this function will load the CSVs as dataframe and ingest into db
    """
    start = time.time()
    for file in os.listdir('data'):
        if '.csv' in file:
            df = pd.read_csv('data/' + file)
            
            # Clean up column names
            df.columns = df.columns.str.strip().str.replace(' ', '_')
            
            logging.info(f'Ingesting {file} in db')
            ingest_db(df, file[:-4], conn)
    end = time.time()
    total_minutes = (end - start) / 60
    logging.info("All CSVs ingested successfully.")
    logging.info(f"Total time taken: {total_minutes} minutes")

if __name__ == '__main__':
    load_raw_data()